In [1]:

import os
import torch
import numpy as np
import pytorch_lightning as pl
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import loggers as pl_loggers
import torch_geometric.transforms as T
import torch_geometric.data as geom_data
from torch.utils.tensorboard import SummaryWriter

%reload_ext autoreload
%autoreload 2

#### Configure device

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != "cpu":
    torch.set_float32_matmul_precision('high')
    
# device = "cpu"
# torch.set_float32_matmul_precision("high")

num_workers = os.cpu_count()
device, num_workers

(device(type='cuda', index=0), 12)

### Directory configuration and load data

In [3]:
cwd = os.getcwd()
tb_logging_dir = os.path.join(cwd, "lightning_logs")
exp_name = "AG-GraphSAGE-transductive"
exp_dir = os.path.join(tb_logging_dir, exp_name)
dataset_dir = os.path.join(cwd, "dataset", "CUHKSZ_AcademicGraph")
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    
from utils.dataset.CUHKSZ_AcademicGraph import CUHKSZ_AcademicGraph
AGDataset = CUHKSZ_AcademicGraph(dataset_dir, with_title=True, with_label=True)

AGDataset[0]

d:\GitHub\GNN-Cora-CUHKSZAG\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph_Rawdata.zip
d:\GitHub\GNN-Cora-CUHKSZAG\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph-rawdata_released


Data(x=[6614, 768], edge_index=[2, 12330], y=[6614], title=[6614], train_mask=[6614], val_mask=[6614], test_mask=[6614])

#### Note that if numebr of epoch is large, the kernel will stuck after training! Need to load the saved model mannually!

In [6]:
from utils.model.RS_GraphSAGE import GraphSage

dataset = AGDataset

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode='min'
)
hparams = {"aggregator_type": 'mean',
           "HIDDEN_DIM": [112, 16],  # size of the embedding
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }  # The number of neighbors in each order of sampling

tb_logger = pl_loggers.TensorBoardLogger(tb_logging_dir, name=exp_name)
 
trainer = pl.Trainer(max_epochs=100,
                     callbacks=[early_stop_callback],
                     logger=tb_logger,
                     log_every_n_steps=1,
                     # accelerator="cpu"
                    # num_sanity_val_steps = 0
                     )

version_dir = os.path.join(
    exp_dir, "version_"+str(trainer.logger.version))
writer_acc = SummaryWriter(log_dir=version_dir)
writer_loss = SummaryWriter(log_dir=version_dir)

checkpoint_dir = os.path.join(version_dir, "checkpoints")
print("Saving checkpoints to", checkpoint_dir)

SAGEmodel = GraphSage(dataset=dataset, input_dim=dataset.num_features, hparams=hparams,
                      writer_acc=writer_acc, writer_loss=writer_loss).to(device)
SAGEmodel.data_processing_transductive() 

trainer.fit(SAGEmodel)
trainer.test(SAGEmodel)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Saving checkpoints to D:\GitHub\aml-project\GNN\zyq\lightning_logs\AG-GraphSAGE-transductive\version_3\checkpoints


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | aggr  | MeanAggregation   | 0     
1 | model | Sequential_4d85c0 | 175 K 
--------------------------------------------
175 K     Trainable params
0         Non-trainable params
175 K     Total params
0.703     Total estimated model params size (MB)


Data Processing Done on: cuda:0


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9868737541849043     │
│         test_loss         │    0.5499117374420166     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5499117374420166, 'test_acc': 0.9868737541849043}]

#### Upload trained model

In [4]:
from utils.model.RS_GraphSAGE import GraphSage
hparams = {"aggregator_type": 'mean',
           "HIDDEN_DIM": [112, 16],  # size of the embedding
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }
dataset = AGDataset
checkpoint_dir = "lightning_logs\\RS_AG_GraphSAGE_transductive\\best\\checkpoints"
checkpoint_file = os.path.join(checkpoint_dir, os.listdir(checkpoint_dir)[0])
Loaded_model = GraphSage.load_from_checkpoint(checkpoint_file, dataset=dataset, input_dim=dataset.num_features, hparams=hparams)
Loaded_model.data_processing_transductive()

Data Processing Done on: cpu


#### To see the mini-batch's number of negative edges:

In [7]:
tmpLoader = Loaded_model.train_dataloader()
sample = next(iter(tmpLoader))
sample

Data(x=[4273, 768], edge_index=[2, 6720], edge_label=[6720], train_mask=[4273], n_id=[4273], e_id=[6720], input_id=[64], batch_size=64)

#### To see the number of training positive edges:

In [8]:
Loaded_model.pos_data["train"]

Data(x=[6614, 768], edge_index=[2, 7397], edge_label=[7397], train_mask=[6614])

#### Recommendation for a given node

In [6]:
import torch.nn.functional as F
def recommendation_by_transductive_model(A, model):
    ## Return recommedation for A in the given graph
    neg_graph = model.neg_data["all"]
    candidate = torch.zeros(neg_graph.num_edges)

    candidate = neg_graph.edge_index[1, (neg_graph.edge_index[0] == A)].cpu()
    candidate.sort()
    num_candidates = len(candidate) 
    u = torch.ones(num_candidates, dtype=torch.int) * A
    v = torch.Tensor(candidate)
    candidate_link = torch.stack((u, v))

    embedding = model(model.pos_data["all"].x, model.pos_data["all"].edge_index)

    scores = np.zeros(candidate.max()+1) - 1e8
    
    scores[v] = F.sigmoid((embedding[u] * embedding[v]).sum(dim=1)).detach().numpy()

    rank_idx = np.argsort(scores)[::-1]
    
    pos_graph = model.pos_data["all"]
    print("Recommendation for node", A, "with title", pos_graph.title[A])
    for k in range(10):
        print(rank_idx[k], scores[rank_idx[k]], pos_graph.title[rank_idx[k]])
        
recommendation_by_transductive_model(1370, Loaded_model)

Recommendation for node 1370 with title Non-linear matrix completion
119 0.998610258102417 R1-PCA: rotational invariant L1-norm principal component analysis for robust subspace factorization
2413 0.9978931546211243 Continuum Isomap for manifold learnings
5235 0.9978923201560974 Theory of Compressive Sensing via ℓ1-Minimization: a Non-RIP Analysis and Extensions
1021 0.9975487589836121 Improved RIP-Based Bounds for Guaranteed Performance of Several Compressed Sensing Algorithms
417 0.9975274205207825 Matrix Completion via Sparse Factorization Solved by Accelerated Proximal Alternating Linearized Minimization
6150 0.9974560141563416 Exactly Robust Kernel Principal Component Analysis
3514 0.9972105622291565 Graph-Laplacian PCA: Closed-Form Solution and Robustness
3828 0.9970922470092773 Exact Low-Rank Matrix Completion from Sparsely Corrupted Entries Via Adaptive Outlier Pursuit
1620 0.9970524311065674 Isometric Embedding and Continuum ISOMAP
5431 0.9966875910758972 Matrix completion by d

In [10]:
Loaded_model.pos_data["all"].test_mask.argwhere()

tensor([[   2],
        [   7],
        [  11],
        ...,
        [6596],
        [6597],
        [6613]])

In [11]:
recommendation_by_transductive_model(7, Loaded_model)

Recommendation for node 7 with title Nonnegative Matrix Factorization and Probabilistic Latent Semantic Indexing: Equivalence Chi-Square Statistic, and a Hybrid Method
119 0.9999995231628418 R1-PCA: rotational invariant L1-norm principal component analysis for robust subspace factorization
254 0.9999991655349731 Bipartite graph partitioning and data clustering
761 0.9999990463256836 Efficient and Robust Feature Selection via Joint ℓ2, 1-Norms Minimization
576 0.999998927116394 Multi-class protein fold recognition using support vector machines and neural networks
6157 0.9999986886978149 Orthogonal nonnegative matrix t-factorizations for clustering
2914 0.9999984502792358 Two-dimensional PCA: a new approach to appearance-based face representation and recognition
3879 0.999997615814209 Robust nonnegative matrix factorization using L21-norm
824 0.9999967813491821 Convex and Semi-Nonnegative Matrix Factorizations
5804 0.9999961853027344 Solving Consensus and Semi-supervised Clustering Probl

In [9]:
recommendation_by_transductive_model(1676, Loaded_model)

Recommendation for node 1676 with title A Semismooth Newton Stochastic Proximal Point Algorithm with Variance Reduction
2499 0.9951659440994263 Approximation Algorithms for Quadratic Programming
6256 0.994754433631897 Error Bounds for Quadratic Systems
5541 0.9947484135627747 A Proximal Alternating Direction Method of Multiplier for Linearly Constrained Nonconvex Minimization
6219 0.9946396946907043 A Unified Convergence Analysis of Block Successive Minimization Methods for Nonsmooth Optimization
1983 0.9944781064987183 Comparison of two binaural beamforming approaches for hearing aids
2794 0.9942250847816467 On the Convergence Rate of Dual Ascent Methods for Linearly Constrained Convex Minimization
1325 0.9940270185470581 Level-set Subdifferential Error Bounds and Linear Convergence of Variable Bregman Proximal Gradient Method
6126 0.993911862373352 A block coordinate descent method of multipliers: Convergence analysis and applications
1389 0.9931164979934692 Error bounds and converge